# Reference
https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.2/casmopt/casmopt_decomp_examples02.htm

In [103]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import log
import shutil
import sys
import os.path
#from pyomo.environ import *
import pyomo.environ as pe
import pyomo.opt as po

In [104]:
m = pe.ConcreteModel()

In [105]:
# Defining data

m.M = pe.RangeSet(1,8)
m.J = pe.RangeSet(1,24)

data = [36, 35, 38, 34, 32, 34, 31, 34]
# Create a dictionary with keys from 1 to 8 and corresponding values from list b
b = {i: data[i-1] for i in m.M}

In [106]:
data = [
    25, 23, 20, 16, 19, 22, 20, 16, 15, 22, 15, 21, 20, 23, 20, 22, 19, 25, 25, 24, 21, 17, 23, 17,
    16, 19, 22, 22, 19, 23, 17, 24, 15, 24, 18, 19, 20, 24, 25, 25, 19, 24, 18, 21, 16, 25, 15, 20,
    20, 18, 23, 23, 23, 17, 19, 16, 24, 24, 17, 23, 19, 22, 23, 25, 23, 18, 19, 24, 20, 17, 23, 23,
    16, 16, 15, 23, 15, 15, 25, 22, 17, 20, 19, 16, 17, 17, 20, 17, 17, 18, 16, 18, 15, 25, 22, 17,
    17, 23, 21, 20, 24, 22, 25, 17, 22, 20, 16, 22, 21, 23, 24, 15, 22, 25, 18, 19, 19, 17, 22, 23,
    24, 21, 23, 17, 21, 19, 19, 17, 18, 24, 15, 15, 17, 18, 15, 24, 19, 21, 23, 24, 17, 20, 16, 21,
    18, 21, 22, 23, 22, 15, 18, 15, 21, 22, 15, 23, 21, 25, 25, 23, 20, 16, 25, 17, 15, 15, 18, 16,
    19, 24, 18, 17, 21, 18, 24, 25, 18, 23, 21, 15, 24, 23, 18, 18, 23, 23, 16, 20, 20, 19, 25, 21
]

# Define the dictionary
c = {}

# Populate the dictionary
index = 0
for i in range(1, 9):
    for j in range(1, 25):
        c[(i, j)] = data[index]
        index += 1  
##################
data = [
    8, 18, 22, 5, 11, 11, 22, 11, 17, 22, 11, 20, 13, 13, 7, 22, 15, 22, 24, 8, 8, 24, 18, 8,
    24, 14, 11, 15, 24, 8, 10, 15, 19, 25, 6, 13, 10, 25, 19, 24, 13, 12, 5, 18, 10, 24, 8, 5,
    22, 22, 21, 22, 13, 16, 21, 5, 25, 13, 12, 9, 24, 6, 22, 24, 11, 21, 11, 14, 12, 10, 20, 6,
    13, 8, 19, 12, 19, 18, 10, 21, 5, 9, 11, 9, 22, 8, 12, 13, 9, 25, 19, 24, 22, 6, 19, 14,
    25, 16, 13, 5, 11, 8, 7, 8, 25, 20, 24, 20, 11, 6, 10, 10, 6, 22, 10, 10, 13, 21, 5, 19,
    19, 19, 5, 11, 22, 24, 18, 11, 6, 13, 24, 24, 22, 6, 22, 5, 14, 6, 16, 11, 6, 8, 18, 10,
    24, 10, 9, 10, 6, 15, 7, 13, 20, 8, 7, 9, 24, 9, 21, 9, 11, 19, 10, 5, 23, 20, 5, 21, 6,
    9, 9, 5, 12, 10, 16, 15, 19, 18, 20, 18, 16, 21, 11, 12, 22, 16, 21, 25, 7, 14, 16, 10
]
# Define the dictionary
a = {}
# Populate the dictionary
index = 0
for i in range(1, 9):
    for j in range(1, 25):
        a[(i, j)] = data[index]
        index += 1

In [107]:
# Initialising
m.b = pe.Param(m.M, initialize=b)
m.c = pe.Param(m.M,m.J, initialize=c)
m.a = pe.Param(m.M,m.J, initialize=a)

In [108]:
# defining variable
m.x = pe.Var(m.M,m.J, domain=pe.Binary)

In [109]:
# objective
obj_expr = sum(m.c[i,j]*m.x[i,j] for i in m.M for j in m.J)
m.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [110]:
# constraints
def kp_rule(m,i):
    return sum(m.a[i,j]*m.x[i,j] for j in m.J) <= m.b[i]
m.kp = pe.Constraint(m.M,rule=kp_rule)    

def mc_rule(m,j):
    return sum(m.x[i,j] for i in m.M) == 1
m.mc = pe.Constraint(m.J,rule=mc_rule)

In [111]:
#Solve and Postprocess
solver = po.SolverFactory('gurobi')
results = solver.solve(m,tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-25
Read LP format model from file /var/folders/kk/630d4n695dd9k2gbbmxbrcbc0000gp/T/tmprj8y6e80.pyomo.lp
Reading time = 0.00 seconds
x1: 32 rows, 192 columns, 384 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M3 Max
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 32 rows, 192 columns and 384 nonzeros
Model fingerprint: 0x4ab700ca
Variable types: 0 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 493.0000000
Presolve time: 0.00s
Presolved: 32 rows, 192 columns, 384 nonzeros
Variable types: 0 continuous, 192 integer (192 binary)
Found heuristic solution: objective 506.0000000

Root relaxation: objective 5

In [112]:
print(pe.value(m.obj))

563.0


In [113]:
for j in m.J:
    for i in m.M:
        if (pe.value(m.x[i,j]) >0):
            print("Job ",j," is done by machine", i,"thus: x[",i,',',j,']=', pe.value(m.x[i,j]))

Job  1  is done by machine 1 thus: x[ 1 , 1 ]= 1.0
Job  2  is done by machine 1 thus: x[ 1 , 2 ]= 1.0
Job  3  is done by machine 6 thus: x[ 6 , 3 ]= 1.0
Job  4  is done by machine 4 thus: x[ 4 , 4 ]= 1.0
Job  5  is done by machine 5 thus: x[ 5 , 5 ]= 1.0
Job  6  is done by machine 2 thus: x[ 2 , 6 ]= 1.0
Job  7  is done by machine 4 thus: x[ 4 , 7 ]= 1.0
Job  8  is done by machine 8 thus: x[ 8 , 8 ]= 1.0
Job  9  is done by machine 3 thus: x[ 3 , 9 ]= 1.0
Job  10  is done by machine 6 thus: x[ 6 , 10 ]= 1.0
Job  11  is done by machine 2 thus: x[ 2 , 11 ]= 1.0
Job  12  is done by machine 7 thus: x[ 7 , 12 ]= 1.0
Job  13  is done by machine 8 thus: x[ 8 , 13 ]= 1.0
Job  14  is done by machine 7 thus: x[ 7 , 14 ]= 1.0
Job  15  is done by machine 5 thus: x[ 5 , 15 ]= 1.0
Job  16  is done by machine 6 thus: x[ 6 , 16 ]= 1.0
Job  17  is done by machine 5 thus: x[ 5 , 17 ]= 1.0
Job  18  is done by machine 2 thus: x[ 2 , 18 ]= 1.0
Job  19  is done by machine 7 thus: x[ 7 , 19 ]= 1.0
Job  20  is